In [2582]:
import plotly
import pandas as pd
import pickle
import numpy as np

In [2583]:
plotly.__version__

'1.9.0'

In [2584]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

In [2585]:
#read in files
df = pd.read_csv("industries_wide.csv")
del df['Unnamed: 0']

#read in number of companies
with open('num_of_companies.pkl','r') as pf:
    num_of_companies = pickle.load(pf)
    
df_counts = pd.read_csv('industry_counts_total.csv', header=0, names=['industry','count'])

#merge number of companies by industry into dataframe
df = df.merge(df_counts, on='industry', how='outer')

In [2586]:
df['total_weeks'] = df['mean_paid'] + df['mean_unpaid'] #create total leave (in weeks) field
df.sort_values(by='total_weeks',inplace=True, na_position='first') #sort by total leave

df.head()

,industry,count_paid,mean_paid,count_unpaid,mean_unpaid,count,total_weeks
11,Electronics Wholesaler,1,6.000000,0,NaN,1,NaN
22,Maritime,0,NaN,1,12.000,1,NaN
29,Public Relations,2,3.500000,0,NaN,2,NaN
3,Auto dealers,2,8.000000,2,3.500,2,11.500000
1,Advertising,15,7.133333,8,4.375,15,11.508333


In [2587]:
#Define function to manipulate count lists
def clean_counts(ct):
    if np.isnan(ct):
        return str(0)
    return str(ct)

#Load paid and unpaid lists
paid = [round(x,1) for x in df['mean_paid'].tolist()]
unpaid = [round(x,1) for x in df['mean_unpaid'].tolist()]
paid_counts = [clean_counts(elem) for elem in df['count_paid'].tolist()]
unpaid_counts = [clean_counts(elem) for elem in df['count_unpaid'].tolist()]

#Define industries
industries = df['industry'].tolist()

In [2588]:
#Define types of leave
leave = ['Paid','Unpaid']

# Define colors for each type of leave
colors = ['#42A5B3','#D15A86']

# Define a trace-generating function (returns a Bar object)
def make_trace(x, count, color, name):
    return Bar(
        y=industries,   # industries on the y-axis
        x=x,        # leave total on x-axis
        name=name,  # label for hover 
        #text=count,
        orientation='h',   # (!) for horizontal bars, default is 'v'
        marker= Marker(
            color=color,        # set bar colors
            line= Line(
                color='grey',  # set bar border color
                width=.2         # set bar border width
            )
        )
    )

# Define annotation-generating function for data source
def make_anno1(text, fontsize, x, y):
    return Annotation(
        text=text,   # annotation text
        xref='paper',  # use paper coordinates
        yref='paper',  #   for both x and y coords
        x=x,           # x and y position 
        y=y,           #   in norm. coord. 
        font=Font(size=fontsize),  # text font size
        showarrow=False,       # no arrow (default is True)
        bgcolor='#F5F3F2',     # light grey background color
        bordercolor='#808080', # white borders
        borderwidth=1.5,         # border width
        borderpad=fontsize     # set border/text space to 1 fontsize
    )

# Define annotation-generating function for count labels
def make_anno2(x, y, text):
    return Annotation(
        text="({0})".format(text),     # text
        showarrow=False, # annotation w/o arrows, default is True
        x=x,               # set x position
        xref='x',          # position text horizontally with x-coords
        xanchor='center',  # x position corresp. to center of text
        yref='y',            # set y position 
        yanchor='top',       # position text vertically with y-coords
        y=y,                 # y position corresp. to top of text
        font=Font(
            color='#262626',  # set font color
            size=12           #   and size   
        )
    )

In [2589]:
data = [make_trace(paid, paid_counts, colors[0], leave[0]),
            make_trace(unpaid, unpaid_counts, colors[1], leave[1])]

In [2590]:
#Define title
title = "<b>Maternity Leave by Industry</b><br> \
Average Min. Paid and Unpaid Weeks from {0} Companies".format(num_of_companies)

bargap = 0.3

# Make Layout object
layout = Layout(
    barmode='stack',  # (!) bars are stacked on this plot
    bargap=bargap,       # (!) spacing (norm. w.r.t axis) between bars
    title=title, # set plot title
    titlefont = dict(size=25),
    showlegend=True,   # keep legend
    xaxis= XAxis(
        title='Weeks of Maternity Leave', # x-axis title 
        #gridcolor=None,  # no grid lines
        showgrid=True,
        gridwidth=2,        # bigger grid lines
        zeroline=False,     # remove thick zero line
        ticks='outside',    # draw ticks outside axes
        autotick=False,     # (!) overwrite default tick options
        tick0=10,       # set first tick
        dtick=10,          # (!) set distance between ticks  
        ticklen=8,          # (!) set tick length
        tickwidth=1       #     and width
    ),
    autosize=False,   # (!) turn off autosize 
    height=900,       # plot's height and
    width=750,        #   width in pixels, as recommended by Sunlight Found.
    margin=Margin( # set frame to plotting area margins
        t=140,     #   top,
        b=125,     #   bottom,
        r=25,      #   right,
        l=200       #   left
    ),
    paper_bgcolor='rgb(233,233,233)',  # set paper (outside plot) 
    plot_bgcolor='rgb(233,233,233)',   #   and plot color to grey
)   
        
# update annotations
annotations = Annotations([
    make_anno1('<b>Source:</b> Fairygodboss.com<br>(Number of companies submitted in parenthesis)', 11, -.35, -0.18)]
 + [make_anno2(x+1.56, .6+i, ct) for x,i,ct in zip(df['total_weeks'],range(len(df)),df['count'])])

In [2591]:
# Make Figure object, 
fig = Figure(data=data, layout=layout)

In [2592]:
# Update figure object with font and annotations
fig['layout'].update(
    font=Font(
        family="Droid Serif, serif",
        color='#635F5D'
    ),
    annotations=annotations
)

In [2593]:
iplot(fig)